In [1]:
import IPython.display as ipd
import numpy as np
import matplotlib.pyplot as plt
import librosa
import librosa.display
import pylab
import utils_spec
import os

In [2]:
# Load metadata and features.
tracks = utils_spec.load('/ubc/cs/research/tracking-raid/candice/project/DreamingInMusic/data/fma_metadata/tracks.csv')
genres = utils_spec.load('/ubc/cs/research/tracking-raid/candice/project/DreamingInMusic/data/fma_metadata/genres.csv')

rockGenreIds = []
hiphopGenreIds = []
popGenreIds = []
jazzGenreIds = []
elecGenreIds = []
classicalGenreIds = []
for key in tracks['track','genres'].keys():
    if len(tracks['track','genres'][key]) >= 1:
        if tracks['track','genres'][key][0] == 12:
            rockGenreIds.append(key)
        if tracks['track','genres'][key][0] == 21:
            hiphopGenreIds.append(key)
        if tracks['track','genres'][key][0] == 10:
            popGenreIds.append(key)
        if tracks['track','genres'][key][0] == 4:
            jazzGenreIds.append(key)
        if tracks['track','genres'][key][0] == 15:
            elecGenreIds.append(key)
        if tracks['track','genres'][key][0] == 5:
            classicalGenreIds.append(key)

('track', 'tags')
('album', 'tags')
('artist', 'tags')
('track', 'genres')
('track', 'genres_all')


In [3]:
def get_filenames(audioDirectory, ids):
    file_list = []
    for count, Id in enumerate(ids):
        try:
            Id = ids[count]
            #print count
            Id = str(Id)
            while(len(Id) < 6):
                Id = "0" + Id
            #print (rockId)
            path = os.path.join(audioDirectory, Id[0:3])
            files = os.listdir(path)
            audioFilename = os.path.join(path, Id + ".mp3")
            if os.path.isfile(audioFilename):
                file_list.append(audioFilename)
        except:
            print count, Id
        
    return file_list

audioDirectory = "/ubc/cs/research/tracking-raid/candice/project/fma_medium"
rock_song_list = get_filenames(audioDirectory, rockGenreIds)
hiphop_song_list = get_filenames(audioDirectory, hiphopGenreIds)
elec_song_list = get_filenames(audioDirectory, elecGenreIds)
classical_song_list = get_filenames(audioDirectory, classicalGenreIds)

In [4]:
print len(rock_song_list)
print len(hiphop_song_list)
elec_song_list = elec_song_list[:2000]
print len(elec_song_list)
print len(classical_song_list)
song_list = rock_song_list + hiphop_song_list + elec_song_list + classical_song_list
print len(song_list)

2406
1957
2000
448
6811


In [5]:
import scipy.misc

minPhase = -3.14
maxPhase = 3.14
def scalePhase(x, new_size):
    x = (x - minPhase)/(maxPhase - minPhase)
    y = scipy.misc.imresize(x, new_size, mode='L', interp='nearest')
    return y
def unscalePhase(x, new_size, minAmp, maxAmp):
    x = scipy.misc.imresize(x, new_size, mode='L', interp='nearest')
    x = minPhase + x*(maxPhase - minPhase)
    return x
def convert_file_to_phase(audio_filename):
    """
    Simple function to load and preprocess the image.

    1. Open the image.
    2. Scale/crop it and convert it to a float tensor.
    3. Convert it to a variable (all inputs to PyTorch models must be variables).
    4. Add another dimension to the start of the Tensor (b/c VGG expects a batch).
    5. Move the variable onto the GPU.
    """
    try:
        x1, sr1 = librosa.load(audio_filename, sr=None, mono=True, duration=9.98, offset=0)
    except:
        return
    try:
            x2, sr2 = librosa.load(audio_filename, sr=None, mono=True, duration=9.98, offset=10)
    except:
        return
    try:
        x3, sr3 = librosa.load(audio_filename, sr=None, mono=True, duration=9.98, offset=20)
    except:
        return
    if x1.shape[0] != 0 and x2.shape[0] != 0 and x3.shape[0] != 0:
        #Convert audio to a complex valued spectrogram
        spectro1 = librosa.core.stft(x1)
        spectro2 = librosa.core.stft(x2)
        spectro3 = librosa.core.stft(x3)
        if spectro1 is not None and spectro2 is not None and spectro3 is not None:
            #Separate out amplitude and phase from complex valued spectrogram
            mag1, phase1 = librosa.core.magphase(spectro1)
            mag2, phase2 = librosa.core.magphase(spectro2)
            mag3, phase3 = librosa.core.magphase(spectro3)

            #Get the decibal version from power spectrogram
            #This is the value that should be stored for training
            phase1 = np.angle(phase1)
            phase2 = np.angle(phase2)
            phase3 = np.angle(phase3)
            return phase1, phase2, phase3
        else:
            return None, None, None

In [6]:
def save_spectros(song_list, genre_name, save_dir):
    num_songs = len(song_list)
    num_val_songs = num_songs / 5
    print num_songs, num_val_songs
    for i, song in enumerate(song_list):
        print i, song
        if i < num_val_songs:
            #print "val"
            save_ddir = os.path.join(save_dir, 'val')
        else:
            #print "train"
            save_ddir = os.path.join(save_dir, 'train')
        if not os.path.exists(save_ddir):
            os.makedirs(save_ddir)
        save_subdir = os.path.join(save_ddir, genre_name)
        if not os.path.exists(save_subdir):
            os.makedirs(save_subdir)
        if convert_file_to_phase(song) is not None:
            phase1, phase2, phase3 = convert_file_to_phase(song)
        else:
            continue
        if phase1 is not None and phase2 is not None and phase3 is not None:
            scaled_phase1 = scalePhase(phase1, (512, 512))
            scaled_phase2 = scalePhase(phase2, (512, 512))
            scaled_phase3 = scalePhase(phase3, (512, 512))
            save_name1 = os.path.join(save_subdir, str(3*i+1) + '.jpg')
            scipy.misc.imsave(save_name1, scaled_phase1)
            save_name2 = os.path.join(save_subdir, str(3*i+2) + '.jpg')
            scipy.misc.imsave(save_name2, scaled_phase2)
            save_name3 = os.path.join(save_subdir, str(3*i+3) + '.jpg')
            scipy.misc.imsave(save_name3, scaled_phase3)
        else:
            continue

In [7]:
save_dir = '/ubc/cs/research/tracking-raid/candice/project/dataset/phase_512'
save_spectros(rock_song_list, 'rock', save_dir)
save_spectros(hiphop_song_list, 'hiphop', save_dir)
save_spectros(classical_song_list, 'classical', save_dir)
save_spectros(elec_song_list, 'elec', save_dir)

2406 481
0 /ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000247.mp3
1 /ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000249.mp3
2 /ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000255.mp3
3 /ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000564.mp3
4 /ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000605.mp3
5 /ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000608.mp3
6 /ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000633.mp3
7 /ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000635.mp3
8 /ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000690.mp3
9 /ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000720.mp3
10 /ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000777.mp3
11 /ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000824.mp3
12 /ubc/cs/research/tracking-raid/candice/project/fma_medium/000/000825.mp3
13 /ubc/cs/re

108 /ubc/cs/research/tracking-raid/candice/project/fma_medium/003/003473.mp3
109 /ubc/cs/research/tracking-raid/candice/project/fma_medium/003/003475.mp3
110 /ubc/cs/research/tracking-raid/candice/project/fma_medium/003/003476.mp3
111 /ubc/cs/research/tracking-raid/candice/project/fma_medium/003/003477.mp3
112 /ubc/cs/research/tracking-raid/candice/project/fma_medium/003/003478.mp3
113 /ubc/cs/research/tracking-raid/candice/project/fma_medium/003/003479.mp3
114 /ubc/cs/research/tracking-raid/candice/project/fma_medium/003/003489.mp3
115 /ubc/cs/research/tracking-raid/candice/project/fma_medium/003/003490.mp3
116 /ubc/cs/research/tracking-raid/candice/project/fma_medium/003/003491.mp3
117 /ubc/cs/research/tracking-raid/candice/project/fma_medium/003/003492.mp3
118 /ubc/cs/research/tracking-raid/candice/project/fma_medium/003/003493.mp3
119 /ubc/cs/research/tracking-raid/candice/project/fma_medium/003/003495.mp3
120 /ubc/cs/research/tracking-raid/candice/project/fma_medium/003/003497.mp3

215 /ubc/cs/research/tracking-raid/candice/project/fma_medium/004/004310.mp3
216 /ubc/cs/research/tracking-raid/candice/project/fma_medium/004/004311.mp3
217 /ubc/cs/research/tracking-raid/candice/project/fma_medium/004/004312.mp3
218 /ubc/cs/research/tracking-raid/candice/project/fma_medium/004/004313.mp3
219 /ubc/cs/research/tracking-raid/candice/project/fma_medium/004/004316.mp3
220 /ubc/cs/research/tracking-raid/candice/project/fma_medium/004/004330.mp3
221 /ubc/cs/research/tracking-raid/candice/project/fma_medium/004/004332.mp3
222 /ubc/cs/research/tracking-raid/candice/project/fma_medium/004/004333.mp3
223 /ubc/cs/research/tracking-raid/candice/project/fma_medium/004/004346.mp3
224 /ubc/cs/research/tracking-raid/candice/project/fma_medium/004/004348.mp3
225 /ubc/cs/research/tracking-raid/candice/project/fma_medium/004/004363.mp3
226 /ubc/cs/research/tracking-raid/candice/project/fma_medium/004/004368.mp3
227 /ubc/cs/research/tracking-raid/candice/project/fma_medium/004/004400.mp3

322 /ubc/cs/research/tracking-raid/candice/project/fma_medium/008/008575.mp3
323 /ubc/cs/research/tracking-raid/candice/project/fma_medium/008/008914.mp3
324 /ubc/cs/research/tracking-raid/candice/project/fma_medium/008/008917.mp3
325 /ubc/cs/research/tracking-raid/candice/project/fma_medium/008/008920.mp3
326 /ubc/cs/research/tracking-raid/candice/project/fma_medium/008/008921.mp3
327 /ubc/cs/research/tracking-raid/candice/project/fma_medium/008/008922.mp3
328 /ubc/cs/research/tracking-raid/candice/project/fma_medium/009/009550.mp3
329 /ubc/cs/research/tracking-raid/candice/project/fma_medium/009/009551.mp3
330 /ubc/cs/research/tracking-raid/candice/project/fma_medium/009/009552.mp3
331 /ubc/cs/research/tracking-raid/candice/project/fma_medium/009/009553.mp3
332 /ubc/cs/research/tracking-raid/candice/project/fma_medium/009/009554.mp3
333 /ubc/cs/research/tracking-raid/candice/project/fma_medium/009/009555.mp3
334 /ubc/cs/research/tracking-raid/candice/project/fma_medium/009/009556.mp3

429 /ubc/cs/research/tracking-raid/candice/project/fma_medium/013/013560.mp3
430 /ubc/cs/research/tracking-raid/candice/project/fma_medium/013/013563.mp3
431 /ubc/cs/research/tracking-raid/candice/project/fma_medium/013/013564.mp3
432 /ubc/cs/research/tracking-raid/candice/project/fma_medium/013/013596.mp3
433 /ubc/cs/research/tracking-raid/candice/project/fma_medium/013/013661.mp3
434 /ubc/cs/research/tracking-raid/candice/project/fma_medium/013/013664.mp3
435 /ubc/cs/research/tracking-raid/candice/project/fma_medium/013/013666.mp3
436 /ubc/cs/research/tracking-raid/candice/project/fma_medium/013/013863.mp3
437 /ubc/cs/research/tracking-raid/candice/project/fma_medium/013/013864.mp3
438 /ubc/cs/research/tracking-raid/candice/project/fma_medium/013/013867.mp3
439 /ubc/cs/research/tracking-raid/candice/project/fma_medium/013/013868.mp3
440 /ubc/cs/research/tracking-raid/candice/project/fma_medium/013/013879.mp3
441 /ubc/cs/research/tracking-raid/candice/project/fma_medium/013/013891.mp3

...

1479 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071198.mp3
1480 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071199.mp3
1481 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071200.mp3
1482 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071201.mp3
1483 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071202.mp3
1484 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071203.mp3
1485 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071313.mp3
1486 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071315.mp3
1487 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071317.mp3
1488 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071318.mp3
1489 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071320.mp3
1490 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071321.mp3
1491 /ubc/cs/research/tracking-raid/candice/project/fma_medium/0

1585 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071960.mp3
1586 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071964.mp3
1587 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071966.mp3
1588 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071986.mp3
1589 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071988.mp3
1590 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071989.mp3
1591 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071990.mp3
1592 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071991.mp3
1593 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071992.mp3
1594 /ubc/cs/research/tracking-raid/candice/project/fma_medium/071/071999.mp3
1595 /ubc/cs/research/tracking-raid/candice/project/fma_medium/072/072004.mp3
1596 /ubc/cs/research/tracking-raid/candice/project/fma_medium/072/072005.mp3
1597 /ubc/cs/research/tracking-raid/candice/project/fma_medium/0

1691 /ubc/cs/research/tracking-raid/candice/project/fma_medium/074/074190.mp3
1692 /ubc/cs/research/tracking-raid/candice/project/fma_medium/074/074367.mp3
1693 /ubc/cs/research/tracking-raid/candice/project/fma_medium/074/074443.mp3
1694 /ubc/cs/research/tracking-raid/candice/project/fma_medium/074/074539.mp3
1695 /ubc/cs/research/tracking-raid/candice/project/fma_medium/074/074540.mp3
1696 /ubc/cs/research/tracking-raid/candice/project/fma_medium/074/074557.mp3
1697 /ubc/cs/research/tracking-raid/candice/project/fma_medium/074/074558.mp3
1698 /ubc/cs/research/tracking-raid/candice/project/fma_medium/074/074559.mp3
1699 /ubc/cs/research/tracking-raid/candice/project/fma_medium/074/074560.mp3
1700 /ubc/cs/research/tracking-raid/candice/project/fma_medium/074/074561.mp3
1701 /ubc/cs/research/tracking-raid/candice/project/fma_medium/074/074562.mp3
1702 /ubc/cs/research/tracking-raid/candice/project/fma_medium/074/074941.mp3
1703 /ubc/cs/research/tracking-raid/candice/project/fma_medium/0

1797 /ubc/cs/research/tracking-raid/candice/project/fma_medium/080/080018.mp3
1798 /ubc/cs/research/tracking-raid/candice/project/fma_medium/080/080022.mp3
1799 /ubc/cs/research/tracking-raid/candice/project/fma_medium/080/080023.mp3
1800 /ubc/cs/research/tracking-raid/candice/project/fma_medium/080/080025.mp3
1801 /ubc/cs/research/tracking-raid/candice/project/fma_medium/080/080027.mp3
1802 /ubc/cs/research/tracking-raid/candice/project/fma_medium/080/080028.mp3
1803 /ubc/cs/research/tracking-raid/candice/project/fma_medium/080/080029.mp3
1804 /ubc/cs/research/tracking-raid/candice/project/fma_medium/080/080030.mp3
1805 /ubc/cs/research/tracking-raid/candice/project/fma_medium/080/080076.mp3
1806 /ubc/cs/research/tracking-raid/candice/project/fma_medium/080/080077.mp3
1807 /ubc/cs/research/tracking-raid/candice/project/fma_medium/080/080078.mp3
1808 /ubc/cs/research/tracking-raid/candice/project/fma_medium/080/080079.mp3
1809 /ubc/cs/research/tracking-raid/candice/project/fma_medium/0

1903 /ubc/cs/research/tracking-raid/candice/project/fma_medium/082/082337.mp3
1904 /ubc/cs/research/tracking-raid/candice/project/fma_medium/082/082440.mp3
1905 /ubc/cs/research/tracking-raid/candice/project/fma_medium/082/082442.mp3
1906 /ubc/cs/research/tracking-raid/candice/project/fma_medium/082/082447.mp3
1907 /ubc/cs/research/tracking-raid/candice/project/fma_medium/082/082452.mp3
1908 /ubc/cs/research/tracking-raid/candice/project/fma_medium/082/082456.mp3
1909 /ubc/cs/research/tracking-raid/candice/project/fma_medium/082/082458.mp3
1910 /ubc/cs/research/tracking-raid/candice/project/fma_medium/082/082459.mp3
1911 /ubc/cs/research/tracking-raid/candice/project/fma_medium/082/082461.mp3
1912 /ubc/cs/research/tracking-raid/candice/project/fma_medium/082/082499.mp3
1913 /ubc/cs/research/tracking-raid/candice/project/fma_medium/082/082501.mp3
1914 /ubc/cs/research/tracking-raid/candice/project/fma_medium/082/082622.mp3
1915 /ubc/cs/research/tracking-raid/candice/project/fma_medium/0